## Finilizing Data Generation for 0D Reactor

In [1]:
### Importing Libraries

import sys
print(sys.version)
import os
import numpy as np
import pandas as pd
import time

import matplotlib.pyplot as plt
WORKSPACE_PATH  = os.environ['WORKSPACE_PATH']
plt.style.use(WORKSPACE_PATH+'/ROMNet/romnet/extra/postprocessing/presentation.mplstyle')

from scipy.integrate import solve_ivp

3.8.11 (default, Jun 29 2021, 03:17:05) 
[Clang 12.0.0 (clang-1200.0.32.29)]


In [2]:
### Input Data

OutputDir  = WORKSPACE_PATH+'/ROMNet/Data/0DReact_Isobaric_100Cases_Highest/'
try:
    os.makedirs(OutputDir)
except OSError as e:
    pass

tStratch           = 1.
SOLVER             = 'BDF'

ValidPerc          = 20.

FixedMinVal        = 1.e-14

In [3]:
FileName = OutputDir + '/Orig/train/ext/Cleanvars.csv'
Vars     = pd.read_csv(FileName, header=None)
Vars     = list(Vars.to_numpy()[0,:])
Vars0    = [Vars[i]+'0' for i in range(len(Vars))]
Var0Dic  = {}
for Var in Vars:
    Var0Dic[Var] = Var+'0'

    
InputVar     = ['t', 'log10(t)', 'log(t)', 'T0', 'P0', 'Phi0'] + Vars0



FileName     = OutputDir + '/Orig/train/ext/SimIdxs.csv'
Data         = pd.read_csv(FileName, header=0)
SimIdxs      = Data.to_numpy(int)


FileName     = OutputDir + '/Orig/train/ext/ICs.csv'
Data         = pd.read_csv(FileName)
ICVecs       = Data.to_numpy()
NICs         = len(ICVecs)


P0Vec        = []
EqRatio0Vec  = []
T0Vec        = []
for iC in range(NICs):
    Nts      = SimIdxs[iC,1]-SimIdxs[iC,0]
    P0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,0])
    EqRatio0Vec.append( np.ones(Nts, dtype=np.float64)*ICVecs[iC,1])
    T0Vec.append(       np.ones(Nts, dtype=np.float64)*ICVecs[iC,2])
P0Vec        = np.concatenate(P0Vec)[...,np.newaxis]
EqRatio0Vec  = np.concatenate(EqRatio0Vec)[...,np.newaxis]
T0Vec        = np.concatenate(T0Vec)[...,np.newaxis]


# for iC in range(NICs):
#     Nts              = SimIdxs[iC,1]-SimIdxs[iC,0]
#     FileName         = OutputDir+'/'+str(NVarsRed)+'PC/train/ext/PC.csv.'+str(iC+1)
#     DataTemp         = pd.read_csv(FileName, header=0)
#     DataTemp         = DataTemp[DataTemp['t'] == 0.][Vars]
#     DataTemp         = DataTemp.loc[DataTemp.index.repeat(Nts)]
#     if (iC==0):
#         Data0        = DataTemp
#     else:
#         Data0        = pd.concat([Data0, DataTemp], axis=0)
# Data0.columns = Vars0
# Data0.reset_index(drop=True, inplace=True)
for iC in range(NICs):
        
    Nts                  = SimIdxs[iC,1]-SimIdxs[iC,0]
    NtsNow               = Nts

    

    FileName             = OutputDir+'/Orig/train/ext/y.csv.'+str(iC+1)
    DataTemp             = pd.read_csv(FileName, header=0)
    tVec                 = DataTemp['t']
    DataTemp             = DataTemp.iloc[[1]][Vars] #DataTemp.iloc[1][Vars]
    DataTemp.rename(columns=Var0Dic, errors="raise")
    tVec[tVec<FixedMinVal] = FixedMinVal
    DataTemp['t']        = tVec
    DataTemp['log(t)']   = np.log(tVec)
    DataTemp['log10(t)'] = np.log10(tVec)
    DataTemp['Phi0']     = ICVecs[iC,1]
    DataTemp['T0']       = ICVecs[iC,2]
    DataTemp['P0']       = ICVecs[iC,0]
    DataTemp             = DataTemp.loc[DataTemp.index.repeat(NtsNow)] # DataTemp.loc[DataTemp.index.repeat(Nts)]
    DataTemp.reset_index(drop=True, inplace=True)
    
    if (iC==0):
        DataInput        = DataTemp
    else:
        DataInput        = pd.concat([DataInput, DataTemp], axis=0)

        

    FileName             = OutputDir+'/Orig/train/ext/y.csv.'+str(iC+1)
    DataTemp             = pd.read_csv(FileName, header=0)
    tVec                 = DataTemp['t']
    tVec[tVec<FixedMinVal] = FixedMinVal
    #DataTemp             = DataTemp[DataTemp.t > FixedMinVal][Vars]
    DataTemp['t']        = tVec
    DataTemp['log(t)']   = np.log(tVec)
    DataTemp['log10(t)'] = np.log10(tVec)
    DataTemp['Phi0']     = np.ones(NtsNow)*ICVecs[iC,1]
    DataTemp['T0']       = np.ones(NtsNow)*ICVecs[iC,2]
    DataTemp['P0']       = np.ones(NtsNow)*ICVecs[iC,0]
    DataTemp.reset_index(drop=True, inplace=True)

    if (iC==0):
        DataOutput        = DataTemp
    else:
        DataOutput        = pd.concat([DataOutput, DataTemp], axis=0)
    
    
    
DataInput.reset_index(drop=True, inplace=True)
DataOutput.reset_index(drop=True, inplace=True)

In [4]:
T0Vec

array([[1675.],
       [1675.],
       [1675.],
       ...,
       [1365.],
       [1365.],
       [1365.]])

In [5]:
### Writing Input
DataInput.to_csv(OutputDir+'/Orig/train/ext/Input.csv', index=False)
tVec                = DataInput.t.to_numpy()

### Writing PCs
DataOutput.to_csv(OutputDir+'/Orig/train/ext/Output.csv', index=False)

Data_MinMax         = pd.concat([DataOutput[Vars].min(), DataOutput[Vars].max()], axis=1)
Data_MinMax.columns = ['Min','Max']
Data_MinMax.to_csv(OutputDir+'/Orig/train/ext/Output_MinMax.csv', index=False)

In [6]:
from sklearn.model_selection import train_test_split

DataInput_pts        = DataInput[DataInput['t'] > FixedMinVal]
DataOutput_pts       = DataOutput[DataOutput['t'] > FixedMinVal]
n_points             = len(DataOutput_pts)

idx                  = np.arange(n_points)
train_idx, valid_idx = train_test_split(idx, test_size=ValidPerc/100, random_state=42)

n_valid              = len(valid_idx)
n_train              = len(train_idx)

try:
    os.makedirs(OutputDir+'/Orig/train/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/')
except:
    pass

In [7]:
data_id    = 'pts'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput_pts.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput_pts.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput_pts.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput_pts.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [8]:
data_id    = 'res'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

DataInput_pts.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
DataInput_pts.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

DataOutput_pts[Vars] = DataOutput_pts[Vars].to_numpy() * 0.
DataOutput_pts.iloc[train_idx].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
DataOutput_pts.iloc[valid_idx].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

<ipython-input-8-c1f16f990038>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DataOutput_pts[Vars] = DataOutput_pts[Vars].to_numpy() * 0.
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.8/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [9]:
data_id = 'ics'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

Data_Input_ics               = DataInput[DataOutput['t'] == FixedMinVal]
Data_Output_ics              = DataOutput[DataOutput['t'] == FixedMinVal]
n_points_ics                 = len(Data_Output_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=41)

Data_Input_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
Data_Input_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

Data_Output_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
Data_Output_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [10]:
data_id = 'scs'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

Mask                         = [i for i in range(1,len(DataOutput)-1) if DataOutput.iloc[i-1]['t'] == FixedMinVal]
Data_Input_ics               = DataInput.iloc[Mask]
Data_Output_ics              = DataOutput.iloc[Mask]
n_points_ics                 = len(Data_Input_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=41)

Data_Input_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
Data_Input_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

Data_Output_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
Data_Output_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [11]:
data_id = 'fcs'

try:
    os.makedirs(OutputDir+'/Orig/train/'+data_id+'/')
except:
    pass
try:
    os.makedirs(OutputDir+'/Orig/valid/'+data_id+'/')
except:
    pass

Mask                         = [i for i in range(-1,len(DataOutput)-1) if DataOutput.iloc[i+1]['t'] == FixedMinVal]
Data_Input_ics               = DataInput.iloc[Mask]
Data_Output_ics              = DataOutput.iloc[Mask]
n_points_ics                 = len(Data_Input_ics)
idx_ics                      = np.arange(n_points_ics)
train_idx_ics, valid_idx_ics = train_test_split(idx_ics, test_size=ValidPerc/100, random_state=43)

Data_Input_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Input.csv', index=False)
Data_Input_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Input.csv', index=False)

Data_Output_ics.iloc[train_idx_ics].to_csv(OutputDir+'/Orig/train/'+data_id+'/Output.csv', index=False)
Data_Output_ics.iloc[valid_idx_ics].to_csv(OutputDir+'/Orig/valid/'+data_id+'/Output.csv', index=False)

In [12]:
# ### Retrieving Data

# Vars = []
# for iVars in range(1,NVarsRed+1):
#     Vars.append('PC_'+str(iVars))

# SVars = []
# for iVars in range(1,NVarsRed+1):
#     SVars.append('SPC_'+str(iVars))

# FileName     = TestDir + '/orig_data/RestVecTot.csv'
# Data         = pd.read_csv(FileName, header=None)
# RestVecTot   = np.log10(Data.to_numpy())
    
# # FileName     = TestDir+'/orig_data/t.csv'
# # Data         = pd.read_csv(FileName, header=None)
# # tVec         = Data.to_numpy()[:,0]

# # FileName     = TestDir + '/orig_data/yCleaned.csv'
# # yMatOrig     = pd.read_csv(FileName, header=0)
# # VarNames     = list(yMatOrig.columns)
# # yMat         = yMatOrig.to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PC.csv'
# Data                 = pd.read_csv(FileName, header=0)
# Data['log10(t)']     = np.log10(Data.t.to_numpy()+1.e-15)
# Data['log10(Rest)']  = RestVecTot
# Data['Rest']         = 10.**RestVecTot
# yMat_pca             = Data[Vars].to_numpy()

# FileName             = TestDir+'/'+str(NVarsRed)+'PC/PCSource.csv'
# DataS                = pd.read_csv(FileName, header=0)
# DataS['log10(t)']    = np.log10(Data.t.to_numpy()+1.e-15)
# DataS['log10(Rest)'] = RestVecTot
# DataS['Rest']        = 10.**RestVecTot
# ySource_pca          = DataS[SVars].to_numpy()


# ### Writing Input
# InputVar     = ['t', 'log10(t)', 'Rest', 'log10(Rest)']
# DataInput    = Data[InputVar]
# DataInput.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Input.csv', index=False)
# tVec         = DataInput.t.to_numpy()


# ### Writing PCs
# Data.to_csv(TestDir+'/'+str(NVarsRed)+'PC/Output.csv', index=False)


# ### Normalizing PCs' Source Terms
# ySourcee_pca = (ySource_pca - yMin)/yRange
# #ySourcee_pca = (ySource_pca - yMin)/yRange[0]

# for iPC in range(NVarsRed):
#     DataS['SPC_'+str(iPC+1)+'_Scaled'] = ySourcee_pca[:,iPC]
# DataS.to_csv(TestDir+'/'+str(NVarsRed)+'PC/dOutput.csv', index=False)